# Imports

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Load Data

In [20]:
data = pd.read_csv('data/DC_Properties.csv')

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Cleaning

## remove rows with missing values

In [21]:
# since our many concern is the price of the building, rows without the price have only a small to none value
data = data[data["PRICE"].notnull()]

# remove rows with strange prices
data = data.loc[(data["PRICE"] <= 25000000) & (data["PRICE"] >= 40000)]

# unqualifed data is not representative for the market
data = data[data.QUALIFIED != "U"]

## drop columns

In [22]:
# see above
data = data.drop(["QUALIFIED"], axis=1)
# this column has only one value "2018-07-22 18:01:43" => useless
data = data.drop(["GIS_LAST_MOD_DTTM"], axis=1)
# since the unique id is equal to the row number we don't need it
data = data.drop(["Unnamed: 0"], axis=1)
# many many missing values
data = data.drop(["LIVING_GBA", "CMPLX_NUM"], axis=1)
# these columns have nothing to say
data = data.drop(["STATE", "CITY", "SOURCE", "BLDG_NUM"], axis=1)
# these columns contain only unique texts which can't be computed
data = data.drop(["NATIONALGRID", "FULLADDRESS"], axis=1)
# in our other notebook we proved that there is a realy high (< 0.999) correlation between x, y and longitude and latititude because in both show the longitude and latitdude of a building
data = data.drop(["X", "Y"], axis=1)
# this column has way to many unique values => makes the onehot encoded data really big. It correlates strongly with the other location information 
data = data.drop(["CENSUS_BLOCK"], axis=1)
# we have to many columns with location information
data = data.drop(["ZIPCODE", "ASSESSMENT_NBHD", "CENSUS_TRACT", "WARD", "SQUARE", "QUADRANT"], axis=1)

## data insertion

In [23]:
data.loc[data['YR_RMDL'].isnull(), 'YR_RMDL'] = data['EYB']
data.loc[data['AYB'].isnull(), 'AYB'] = data['EYB']
# set the missing sale date to the year it was build
data["SALEDATE"] = np.where(data["SALEDATE"].isnull(), data["AYB"], data["SALEDATE"])
# set missing values with the mean, mode
data.NUM_UNITS[data.NUM_UNITS.isnull()] =  data.NUM_UNITS.mode().iloc[0]
data.STORIES=data.STORIES.round()
data.STORIES[data.STORIES.isnull()] =  data.STORIES.mode().iloc[0]
data.GBA[data.GBA.isnull()] =  data.GBA.mean()
data.STYLE[data.STYLE.isnull()] =  data.STYLE.mode().iloc[0]
data.STRUCT[data.STRUCT.isnull()] =  data.STRUCT.mode().iloc[0]
data.GRADE[data.GRADE.isnull()] =  data.GRADE.mode().iloc[0]
data.CNDTN[data.CNDTN.isnull()] =  data.CNDTN.mode().iloc[0]
data.EXTWALL[data.EXTWALL.isnull()] =  data.EXTWALL.mode().iloc[0]
data.ROOF[data.ROOF.isnull()] =  data.ROOF.mode().iloc[0]
data.INTWALL[data.INTWALL.isnull()] =  data.INTWALL.mode().iloc[0]
data.KITCHENS[data.KITCHENS.isnull()] =  data.KITCHENS.mode().iloc[0]

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [24]:
# these values make no sense
data.ROOMS = data.ROOMS.astype(int)
data = data[data.ROOMS != 0]
data = data[data.AC != "0"]
data.STORIES = data.STORIES.astype(int)
data = data[data.STORIES < 50]
data = data[data.FIREPLACES < 20]
data = data[data.FIREPLACES < 40]
data = data[data.ASSESSMENT_SUBNBHD.notnull()]

## data convertion

In [25]:
#convert saledate to datetime
data['SALEDATE'] = pd.to_datetime(data['SALEDATE'])
#Calculating the difference in years between Last Sale Date and Year Built
data['SalevYB']=data['SALEDATE'].dt.year - data['AYB']
#Calculating the difference in years between Last Sale Date and Year Improved
data['SalevYI']=data['SALEDATE'].dt.year - data['EYB']

data['SalevYR']=data['SALEDATE'].dt.year - data['YR_RMDL']

data = data.drop(["SALEDATE", "EYB", "AYB", "YR_RMDL"], axis=1)

## final results

In [26]:
missing_values = data.isnull().sum()
print("All Data:")
print("Row Count: " + str(data.shape[0]))
print("Col Count: " + str(data.shape[1]))
print()
print(missing_values)

All Data:
Row Count: 59874
Col Count: 28

BATHRM                0
HF_BATHRM             0
HEAT                  0
AC                    0
NUM_UNITS             0
ROOMS                 0
BEDRM                 0
STORIES               0
PRICE                 0
SALE_NUM              0
GBA                   0
STYLE                 0
STRUCT                0
GRADE                 0
CNDTN                 0
EXTWALL               0
ROOF                  0
INTWALL               0
KITCHENS              0
FIREPLACES            0
USECODE               0
LANDAREA              0
LATITUDE              0
LONGITUDE             0
ASSESSMENT_SUBNBHD    0
SalevYB               0
SalevYI               0
SalevYR               0
dtype: int64


In [27]:
data.describe()

,BATHRM,HF_BATHRM,NUM_UNITS,ROOMS,BEDRM,STORIES,PRICE,SALE_NUM,GBA,KITCHENS,FIREPLACES,USECODE,LANDAREA,LATITUDE,LONGITUDE,SalevYB,SalevYI,SalevYR
count,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,5.987400e+04,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000,59874.000000
mean,1.930053,0.481879,1.139977,6.117129,2.731002,2.062197,5.420962e+05,1.936283,1709.249545,1.161055,0.496342,14.369810,2083.298744,38.912689,-77.014304,69.980359,44.854344,16.855530
std,0.983598,0.575252,0.492488,2.574053,1.354157,0.346256,4.797784e+05,1.469934,633.601846,0.508770,0.818503,3.836785,2483.519796,0.029545,0.039694,36.920528,28.458298,23.318441
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000e+04,1.000000,252.000000,0.000000,0.000000,11.000000,0.000000,38.819953,-77.113909,-11.000000,-21.000000,-26.000000
25%,1.000000,0.000000,1.000000,4.000000,2.000000,2.000000,2.680000e+05,1.000000,1360.000000,1.000000,0.000000,11.000000,581.000000,38.894615,-77.037668,50.000000,30.000000,1.000000
50%,2.000000,0.000000,1.000000,6.000000,3.000000,2.000000,4.300000e+05,1.000000,1730.484413,1.000000,0.000000,13.000000,1463.000000,38.913210,-77.018128,78.000000,44.000000,7.000000
75%,2.000000,1.000000,1.000000,7.000000,3.000000,2.000000,6.850000e+05,3.000000,1730.484413,1.000000,1.000000,16.000000,2520.000000,38.931395,-76.988052,97.000000,55.000000,32.000000
max,11.000000,5.000000,6.000000,25.000000,20.000000,20.000000,2.200000e+07,15.000000,10419.000000,6.000000,13.000000,117.000000,155905.000000,38.982532,-76.909758,262.000000,195.000000,1995.000000


## save to csv

In [28]:
data.to_csv("data/data_cleaned.csv", index=False)